In [20]:
import cv2
import numpy as np

In [21]:
translation_matches = [
    [(300, 300), (400, 500)]
]

translation_matches_test = [
    [(401, 300), (501, 500)]
]

euclidean_matches = [
    [(487, 300), (555, 25)], 
    [(500, 319), (575, 35)]
]

euclidean_matches_test = [
    [(434, 300), (510, 50)]
]

affine_matches = [
    [(300, 300), (400, 350)],
    [(401, 300), (501, 350)],
    [(300, 302), (398, 351)]
]

affine_matches_test = [
    [(500, 302), (598, 351)]
]

projection_matches = [
    [(318, 256), (141, 131)],
    [(534, 372), (480, 159)],
    [(316, 670), (493, 630)],
    [(73, 473), (64, 601)]
]

In [100]:
def get_translation_matrix(point_pairs):

    a = point_pairs[0][1][0] - point_pairs[0][0][0]
    b = point_pairs[0][1][1] - point_pairs[0][0][1]

    transition_matrix = np.array([[1, 0, a],
                                   [0, 1, b],
                                   [0, 0, 1]])

    return transition_matrix

def get_euclidean_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for pair in point_pairs:
        x, y = pair[0]
        x_, y_ = pair[1]

        solution_matrix += [[x, -1 * y, 1, 0],
                            [y,      x, 0, 1]]
        solution_vector += [x_, y_]

    solution_matrix = np.array(solution_matrix)
    solution_vector = np.array(solution_vector)

    a, b, c, d = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, -b, c],
                                  [b,  a, d],
                                  [0,  0, 1]])
    return transition_matrix

def get_affine_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for pair in point_pairs:
        x, y = pair[0]
        x_, y_ = pair[1]

        solution_matrix += [[x, y, 1, 0, 0, 0],
                            [0, 0, 0, x, y, 1]]
        solution_vector += [x_, y_]

    solution_matrix = np.array(solution_matrix)
    solution_vector = np.array(solution_vector)

    a, b, c, d, e, f = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, b, c],
                                  [d, e, f],
                                  [0,  0, 1]])
    return transition_matrix

def get_projection_matrix(point_pairs):

    solution_matrix = []
    solution_vector = []
    for index, pair in enumerate(point_pairs):
        x, y = pair[0]
        x_, y_ = pair[1]

        z_x = [0, 0, 0, 0]
        z_y = [0, 0, 0, 0]
        z_1 = [0, 0, 0, 0]
        z_x[index], z_y[index], z_1[index] = -x_, -y_, -1

        solution_matrix += [[x, y, 1, 0, 0, 0, 0, 0] + z_x,
                            [0, 0, 0, x, y, 1, 0, 0] + z_y,
                            [0, 0, 0, 0, 0, 0, x, y] + z_1]
        solution_vector += [0, 0, -1]

    solution_matrix = np.array(solution_matrix)
    solution_vector = np.array(solution_vector)

    a, b, c, d, e, f, g, h, _, _, _, _ = np.linalg.solve(solution_matrix, solution_vector)

    transition_matrix = np.array([[a, b, c],
                                  [d, e, f],
                                  [g, h, 1]])
    return transition_matrix

def get_transition_matrix(point_pairs):

    if len(point_pairs) == 1:
        return get_translation_matrix(point_pairs)
    elif len(point_pairs) == 2:
        return get_euclidean_matrix(point_pairs)
    elif len(point_pairs) == 3:
        return get_affine_matrix(point_pairs)
    elif len(point_pairs) == 4:
        return get_projection_matrix(point_pairs)
    else:
        raise Exception(f'Received {len(point_pairs)}. get_transition matrix requires 1, 2, 3, or 4 pairs of points.')

In [101]:
def convert_vector(x):

    x = list(x) + [1]
    return np.array(x)

In [114]:
def test_transition_matrix(transition_matrix, pairs):
    
    result = True
    for pair in pairs: 
        x, b = map(convert_vector, pair)

        Ax = np.matmul(transition_matrix, x)
        # Divide by z_ to get back to (x_, y_, 1) space.
        Ax = np.rint(Ax / Ax[2])
        result *= (Ax == b).all()

    return result

In [115]:
t = get_transition_matrix(affine_matches)
t

array([[ 1.00000000e+00, -1.00000000e+00,  4.00000000e+02],
       [ 1.41754162e-16,  5.00000000e-01,  2.00000000e+02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [116]:
T = get_transition_matrix(projection_matches)
T

array([[ 1.80258009e+00,  1.55276411e+00, -7.76398173e+02],
       [-8.09994414e-01,  2.13295256e+00, -1.07909987e+02],
       [ 5.79313440e-04,  7.57830547e-04,  1.00000000e+00]])

In [117]:
test_transition_matrix(T, projection_matches)

[141. 131.   1.] [318 256   1] [141 131   1]
[480. 159.   1.] [534 372   1] [480 159   1]
[493. 630.   1.] [316 670   1] [493 630   1]
[ 64. 601.   1.] [ 73 473   1] [ 64 601   1]


True

In [93]:
test_transition_matrix(t, affine_matches_test + affine_matches)

True

In [69]:
(np.array([400., 350., 1.]) == np.array([400, 350, 1])).all()

True

1

In [33]:
affine_matches_test

[[(500, 302), (598, 351)]]

In [22]:
simple_img = cv2.imread('part2-images/Simple.jpg')

In [23]:
def Apply_Transformation_Two(img, transform_array):

    # Initialize new image
    new_img = np.zeros(shape=img.shape)

    # Loop through coordinates and
    # apply inverse transformation
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            current_coor = np.array([[c, r, 1]])
            new_coor = np.matmul(current_coor, transform_array)
            new_x = new_coor[0] / new_coor[2]
            new_y = new_coor[1] / new_coor[2]
            # Update new image if old coordinate
            # is within the image coordinate bounds
            if new_y < img.shape[0] and \
                new_x < img.shape[1] and \
                new_y > 0 and \
                new_x > 0:

                    new_img[r, c] = img[int(new_y), int(new_x)]

    return new_img

In [24]:
translation_input = np.array([
    [translation_matches[0][0][0], 0],
    [0, translation_matches[0][0][1]]
])

print('Linear Equation Array:')
print(translation_input)
print()

translation_output = np.array([
    translation_matches[0][1][0], translation_matches[0][1][1]
])

print('Equation Output:')
print(translation_output)
print()

translation_solved = np.linalg.solve(translation_input, translation_output)

print('Equation Solved:')
print(translation_solved)
print()

translation_transform = np.array([
    [translation_solved[0], 0, 0],
    [0, translation_solved[1], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(translation_transform)
print()

translation_original = np.array([
    translation_matches_test[0][0][0], translation_matches_test[0][0][1], 1
])

translation_new = np.matmul(translation_original, translation_transform)

print(f'Test: ({translation_new[0] / translation_new[2]}, {translation_new[1] / translation_new[2]}) = {translation_matches_test[0][1]}')

Linear Equation Array:
[[300   0]
 [  0 300]]

Equation Output:
[400 500]

Equation Solved:
[1.33333333 1.66666667]

Transfrom Matrix:
[[1.33333333 0.         0.        ]
 [0.         1.66666667 0.        ]
 [0.         0.         1.        ]]

Test: (534.6666666666666, 500.0) = (501, 500)


In [25]:
euclidean_input = np.array([
    [euclidean_matches[0][0][0], euclidean_matches[0][0][1], 0, 0],
    [0, 0, euclidean_matches[0][0][0], euclidean_matches[0][0][1]],
    [euclidean_matches[1][0][0], euclidean_matches[1][0][1], 0, 0],
    [0, 0, euclidean_matches[1][0][0], euclidean_matches[1][0][1]],
])

print('Linear Equation Array:')
print(euclidean_input)
print()

euclidean_output = np.array([
    euclidean_matches[0][1][0], euclidean_matches[0][1][1], euclidean_matches[1][1][0], euclidean_matches[1][1][1]
])

print('Equation Output:')
print(euclidean_output)
print()

euclidean_solved = np.linalg.solve(euclidean_input, euclidean_output)

print('Equation Solved:')
print(euclidean_solved)
print()

euclidean_transform = np.array([
    [euclidean_solved[0], euclidean_solved[1], 0],
    [euclidean_solved[2], euclidean_solved[3], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(euclidean_transform)
print()

euclidean_original = np.array([
    [euclidean_matches_test[0][0][0], euclidean_matches_test[0][0][1], 1],
])

euclidean_new = np.matmul(euclidean_original[0], euclidean_transform)

print(f'Test: ({euclidean_new[0] / euclidean_new[2]}, {euclidean_new[1] / euclidean_new[2]}) = {euclidean_matches_test[0][1]}')

Linear Equation Array:
[[487 300   0   0]
 [  0   0 487 300]
 [500 319   0   0]
 [  0   0 500 319]]

Equation Output:
[555  25 575  35]

Equation Solved:
[ 0.8490566   0.47169811 -0.47169811  0.8490566 ]

Transfrom Matrix:
[[ 0.8490566   0.47169811  0.        ]
 [-0.47169811  0.8490566   0.        ]
 [ 0.          0.          1.        ]]

Test: (226.98113207547345, 459.43396226414825) = (510, 50)


In [26]:
affine_input = np.array([
    [affine_matches[0][0][0], affine_matches[0][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[0][0][0], affine_matches[0][0][1], 1],
    [affine_matches[1][0][0], affine_matches[1][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[1][0][0], affine_matches[1][0][1], 1],
    [affine_matches[2][0][0], affine_matches[2][0][1], 1, 0, 0, 0],
    [0, 0, 0, affine_matches[2][0][0], affine_matches[2][0][1], 1],
])

print('Linear Equation Array:')
print(affine_input)
print()

affine_output = np.array([
    affine_matches[0][1][0], affine_matches[0][1][1], affine_matches[1][1][0], affine_matches[1][1][1], affine_matches[2][1][0], affine_matches[2][1][1]
])

print('Equation Output:')
print(affine_output)
print()

affine_solved = np.linalg.solve(affine_input, affine_output)

print('Equation Solved:')
print(affine_solved)
print()

affine_transform = np.array([
    [affine_solved[0], affine_solved[1], 0],
    [affine_solved[2], affine_solved[3], 0],
    [0, 0, 1]
])

print('Transfrom Matrix:')
print(affine_transform)
print()

affine_original = np.array([
    [affine_matches_test[0][0][0], affine_matches_test[0][0][1], 1],
])

affine_new = np.matmul(affine_original[0], affine_transform)

print(f'Test: ({affine_new[0] / affine_new[2]}, {affine_new[1] / affine_new[2]}) = {affine_matches_test[0][1]}')

Linear Equation Array:
[[300 300   1   0   0   0]
 [  0   0   0 300 300   1]
 [401 300   1   0   0   0]
 [  0   0   0 401 300   1]
 [300 302   1   0   0   0]
 [  0   0   0 300 302   1]]

Equation Output:
[400 350 501 350 398 351]

Equation Solved:
[ 1.00000000e+00 -1.00000000e+00  4.00000000e+02  1.41754162e-16
  5.00000000e-01  2.00000000e+02]

Transfrom Matrix:
[[ 1.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 4.00000000e+02  1.41754162e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Test: (121300.00000000065, -500.0000000000037) = (598, 351)
